Preprocessing Anime dataset from Kaggle

In [2]:
from collections import defaultdict
import pandas as pd
import numpy as np
import sklearn as sk
import matplotlib as plt

Data preprocessing: Remove unwanted columns from all files

In [3]:
anime_cleaned_df = pd.read_csv("anime_cleaned.csv")
anime_cleaned_df.drop(columns=["title_synonyms", "image_url", "broadcast", "opening_theme", "ending_theme", 
"duration_min", "members", "related", "airing", "aired", "popularity", "favorites", "background"], 
axis = 1, inplace = True)
# anime_cleaned_df.head(20)

users_cleaned_df = pd.read_csv("users_cleaned.csv")
users_cleaned_df = pd.read_csv("users_cleaned.csv")
users_cleaned_df = pd.read_csv("users_cleaned.csv")
users_cleaned_df.head(20)


,username,user_id,user_watching,user_completed,user_onhold,user_dropped,user_plantowatch,user_days_spent_watching,gender,location,birth_date,access_rank,join_date,last_online,stats_mean_score,stats_rewatched,stats_episodes
0,karthiga,2255153,3,49,1,0,0,55.091667,Female,"Chennai, India",1990-04-29 00:00:00,NaN,2013-03-03 00:00:00,2014-02-04 01:32:00,7.43,0.0,3391
1,Damonashu,37326,45,195,27,25,59,82.574306,Male,"Detroit,Michigan",1991-08-01 00:00:00,NaN,2008-02-13 00:00:00,2017-07-10 06:52:54,6.15,6.0,4903
2,bskai,228342,25,414,2,5,11,159.483333,Male,"Nayarit, Mexico",1990-12-14 00:00:00,NaN,2009-08-31 00:00:00,2014-05-12 16:35:00,8.27,1.0,9701
3,terune_uzumaki,327311,5,5,0,0,0,11.394444,Female,"Malaysia, Kuantan",1998-08-24 00:00:00,NaN,2010-05-10 00:00:00,2012-10-18 19:06:00,9.70,6.0,697
4,Bas_G,5015094,35,114,6,20,175,30.458333,Male,"Nijmegen, Nederland",1999-10-24 00:00:00,NaN,2015-11-26 00:00:00,2018-05-10 20:53:37,7.86,0.0,1847
5,HimeAria,3129315,2,87,2,0,28,19.963194,Female,Poland,1996-09-26 00:00:00,NaN,2013-09-08 00:00:00,2018-01-14 13:05:42,8.23,0.0,1314
6,Skallington,326733,0,3,0,0,0,1.234722,Male,Norberg Sweden,1991-03-15 00:00:00,NaN,2010-05-09 00:00:00,2010-05-12 06:29:00,10.00,7.0,73
7,Slimak,61677,79,224,0,3,84,124.731944,Male,Poland,1988-02-21 00:00:00,NaN,2008-05-18 00:00:00,2018-05-01 12:05:21,7.77,2.0,7447
8,Elysiun,5927342,7,4,0,0,4,2.057639,Male,Nebraska,1984-11-16 00:00:00,NaN,2016-12-25 00:00:00,2017-03-11 21:37:00,7.40,0.0,121
9,jimothy000,47167,6,0,0,0,2,2.040972,Male,"England,Manchester",1995-04-25 00:00:00,NaN,2008-04-01 00:00:00,2008-04-27 16:00:00,9.33,5.0,126
